# 创建视图

In [12]:
%%bash
sqlstat="select * from  \
player limit 5;"
echo "$sqlstat"
mysql -e"use nba;$sqlstat"

select * from  player limit 5;
player_id	team_id	player_name	height
10001	1001	韦恩-艾灵顿	1.93
10002	1001	雷吉-杰克逊	1.91
10003	1001	安德烈-德拉蒙德	2.11
10004	1001	索恩-马克	2.16
10005	1001	布鲁斯-布朗	1.96


In [20]:
%%bash
sqlstat="CREATE VIEW player_above_avg_height AS \
SELECT player_id, height \
FROM player \
WHERE height > (SELECT AVG(height) from player)"
mysql -e"use nba;$sqlstat"

# 查询视图


In [21]:
!mysql -e"use nba;show table status where comment='view';"

+-------------------------+--------+---------+------------+------+----------------+-------------+-----------------+--------------+-----------+----------------+-------------+-------------+------------+-----------+----------+----------------+---------+
| Name                    | Engine | Version | Row_format | Rows | Avg_row_length | Data_length | Max_data_length | Index_length | Data_free | Auto_increment | Create_time | Update_time | Check_time | Collation | Checksum | Create_options | Comment |
+-------------------------+--------+---------+------------+------+----------------+-------------+-----------------+--------------+-----------+----------------+-------------+-------------+------------+-----------+----------+----------------+---------+
| player_above_avg_height | NULL   |    NULL | NULL       | NULL |           NULL |        NULL |            NULL |         NULL |      NULL |           NULL | NULL        | NULL        | NULL       | NULL      |     NULL | NULL           | VIEW  

In [22]:
!mysql -e"use nba;SELECT * FROM player_above_avg_height"

+-----------+--------+
| player_id | height |
+-----------+--------+
|     10003 |   2.11 |
|     10004 |   2.16 |
|     10009 |   2.11 |
|     10010 |   2.08 |
|     10011 |   2.08 |
|     10012 |   2.01 |
|     10014 |   2.01 |
|     10015 |   2.11 |
|     10018 |   2.03 |
|     10022 |   2.03 |
|     10023 |   2.11 |
|     10024 |   2.11 |
|     10025 |   2.03 |
|     10029 |   2.03 |
|     10032 |   2.08 |
|     10033 |   2.08 |
|     10036 |   2.06 |
|     10037 |   2.08 |
+-----------+--------+


# 嵌套视图

In [23]:
%%bash
sqlstat="CREATE VIEW player_above_above_avg_height AS \
SELECT player_id, height \
FROM player \
WHERE height > (SELECT AVG(height) from player_above_avg_height)"
mysql -e"use nba;$sqlstat"

In [25]:
!mysql -e"use nba;show table status where comment='view';"

+-------------------------------+--------+---------+------------+------+----------------+-------------+-----------------+--------------+-----------+----------------+-------------+-------------+------------+-----------+----------+----------------+---------+
| Name                          | Engine | Version | Row_format | Rows | Avg_row_length | Data_length | Max_data_length | Index_length | Data_free | Auto_increment | Create_time | Update_time | Check_time | Collation | Checksum | Create_options | Comment |
+-------------------------------+--------+---------+------------+------+----------------+-------------+-----------------+--------------+-----------+----------------+-------------+-------------+------------+-----------+----------+----------------+---------+
| player_above_above_avg_height | NULL   |    NULL | NULL       | NULL |           NULL |        NULL |            NULL |         NULL |      NULL |           NULL | NULL        | NULL        | NULL       | NULL      |     NULL |

In [30]:
!mysql -e"use nba;SELECT * FROM player_above_above_avg_height"

+-----------+--------+
| player_id | height |
+-----------+--------+
|     10003 |   2.11 |
|     10004 |   2.16 |
|     10009 |   2.11 |
|     10010 |   2.08 |
|     10011 |   2.08 |
|     10015 |   2.11 |
|     10023 |   2.11 |
|     10024 |   2.11 |
|     10032 |   2.08 |
|     10033 |   2.08 |
|     10037 |   2.08 |
+-----------+--------+


# 修改视图

In [29]:
%%bash
#增加name
sqlstat="ALTER VIEW player_above_avg_height AS \
SELECT player_id, player_name, height \
FROM player \
WHERE height > (SELECT AVG(height) from player)"
mysql -e"use nba;$sqlstat"
mysql -e"use nba;select * from player_above_avg_height limit 3"

player_id	player_name	height
10003	安德烈-德拉蒙德	2.11
10004	索恩-马克	2.16
10009	扎扎-帕楚里亚	2.11


# 使用视图完成复杂连接

In [32]:
%%bash
sqlstat="CREATE VIEW player_height_grades AS \
SELECT p.player_name, p.height, h.height_level \
FROM player as p JOIN height_grades as h \
ON height BETWEEN h.height_lowest AND h.height_highest"
mysql -e"use nba;$sqlstat"

In [33]:
!mysql -e"use nba;SELECT * FROM player_height_grades WHERE height >= 1.90 AND height <= 2.08;"

+------------------------------------+--------+--------------+
| player_name                        | height | height_level |
+------------------------------------+--------+--------------+
| 韦恩-艾灵顿                        |   1.93 | B            |
| 雷吉-杰克逊                        |   1.91 | B            |
| 布鲁斯-布朗                        |   1.96 | B            |
| 格伦-罗宾逊三世                    |   1.98 | B            |
| 乔恩-洛伊尔                        |   2.08 | A            |
| 布雷克-格里芬                      |   2.08 | A            |
| 雷吉-巴洛克                        |   2.01 | A            |
| 卢克-肯纳德                        |   1.96 | B            |
| 斯坦利-约翰逊                      |   2.01 | A            |
| 凯里-托马斯                        |   1.91 | B            |
| 何塞-卡尔德隆                      |   1.91 | B            |
| 斯维亚托斯拉夫-米凯卢克            |   2.03 | A            |
| 扎克-洛夫顿                        |   1.93 | B            |
| 维克多-奥拉迪波                    |   1.93 | B            |
| 博扬-博格达诺维奇 

# 利用视图对数据进行格式化

In [34]:
%%bash
sqlstat="CREATE VIEW player_team AS \
SELECT CONCAT(player_name, '(' , team.team_name , ')') AS player_team \
FROM player JOIN team WHERE player.team_id = team.team_id"
mysql -e"use nba;$sqlstat"

In [36]:
!mysql -e"use nba;select * from player_team limit 3"

+-----------------------------------------+
| player_team                             |
+-----------------------------------------+
| 韦恩-艾灵顿(底特律活塞)                 |
| 雷吉-杰克逊(底特律活塞)                 |
| 安德烈-德拉蒙德(底特律活塞)             |
+-----------------------------------------+


# 使用视图与计算字段

In [37]:
%%bash
#统计2分球、3分球视图
sqlstat="CREATE VIEW game_player_score AS \
SELECT game_id, player_id, (shoot_hits-shoot_3_hits)*2 AS shoot_2_points,\
shoot_3_hits*3 AS shoot_3_points, shoot_p_hits AS shoot_p_points, score  FROM player_score"
mysql -e"use nba;$sqlstat"